In [1]:
# Import necessary libraries
import pandas as pd
from pymongo import MongoClient

In [7]:
nodes_path = "C:/Users/bwc07/Downloads/nodes_test.tsv"
data = []   # Holds each line seperated in an array
temp = []   # Used to hold line after split by '\t'

# Read the file by each line
with open(nodes_path, "r") as file:
    for line in file:
        line = line[:-1]   #Remove '\n' at end of line
        temp = line.split('\t')
        data.append(temp)
        
# Make dataframe
df_nodes = pd.DataFrame(data)
df_nodes = df_nodes.drop(df_nodes.columns[-1], axis=1)   # Drop last column
df_nodes.columns = df_nodes.iloc[0]   # Make first row the Column titles
df_nodes = df_nodes[1:]   # Remove the first column
df_nodes.head()

,id,name,kind
1,Anatomy::UBERON:0000002,uterine cervix,Anatomy
2,Anatomy::UBERON:0000004,nose,Anatomy
3,Anatomy::UBERON:0000006,islet of Langerhans,Anatomy
4,Anatomy::UBERON:0000007,pituitary gland,Anatomy
5,Anatomy::UBERON:0000010,peripheral nervous system,Anatomy


In [2]:
client = MongoClient('localhost', 27017)  # Change the host and port if necessary
db = client['CSCI49376']  
collection = db['Project1']  

In [11]:
# Convert df of nodes to dictionary and then upload to MongoDB in a collection
nodes_dict = df_nodes.to_dict(orient='records')
collection.insert_many(nodes_dict)


In [13]:
edges_path = "C:/Users/bwc07/Downloads/edges_test.tsv"
data_e = []   # Holds each line seperated in an array
temp_e = []   # Used to hold line after split by '\t'

# Read the file by each line
with open(edges_path, "r") as file:
    for line in file:
        line = line[:-1]   #Remove '\n' at end of line
        temp_e = line.split('\t')
        data_e.append(temp_e)
        
# Make dataframe
df_edges = pd.DataFrame(data_e)
df_edges = df_edges.drop(df_edges.columns[-1], axis=1)   # Drop last column
df_edges.columns = df_edges.iloc[0]   # Make first row the Column titles
df_edges = df_edges[1:]   # Remove the first column

# I have to do this because there was a format error when reading the data
temp_split = df_edges.iloc[409606][0].split(" ")
# Correct data
df_edges['ource'] = df_edges['ource'].replace([df_edges.iloc[409606][0]], temp_split[0])
df_edges['metaedge'] = df_edges['metaedge'].replace(["Gene::10000000"], temp_split[1])
df_edges['target'] = df_edges['target'].replace([None], "Gene::10000000")

df_edges

,ource,metaedge,target
1,Gene::801,GiG,Gene::7428
2,Gene::5987,GiG,Gene::9412
3,Gene::5747,GiG,Gene::79738
4,Gene::3725,GiG,Gene::10514
5,Gene::10014,GiG,Gene::55844
...,...,...,...
1292206,Anatomy::UBERON:0000057,AeG,Gene::65009
1292207,Anatomy::UBERON:0000474,AeG,Gene::80279
1292208,Anatomy::UBERON:0002048,AeG,Gene::1211
1292209,Anatomy::UBERON:0002048,AeG,Gene::8843


In [42]:
relations = ['GiG', 'CrC', 'DdG', 'DlA', 'CtD', 'CbG', 'CuG', 'CdG', 'DrD', 
             'DaG', 'CpD', 'AdG', 'AuG', 'GcG', 'Gr>G', 'DuG', 'AeG']

# GiG = Gene interacts with Gene
# CrC = Compound resembles Compound
# DdG = Disease downregulates Gene
# DlA = Disease localizes Anatomy
# CtD = Compound treats Disease
# CbG = Compound binds to Gene
# CuG = Compound upregulates Gene
# CdG = Compound downregulates Gene
# DrD = Disease resembles Disease
# DaG = Disease associates Gene
# CpD = Compound palliates Disease
# AdG = Anatomy downregulates Gene
# AuG = Anatomy upregulates Gene
# Gene::10000000
# GcG = Gene covaries Gene
# Gr>G = Gene regulates Gene
# DuG = Disease upregulates Gene
# AeG = Anatomy expresses Gene

# All relationships
Relationships = ['UPREGULATES', 'DOWNREGULATES', 'EXPRESSES', 'INTERACTS', 'COVARIES', 
'REGULATES', 'ASSOCIATES', 'BINDS', 'RESEMBLES', 'TREATS', 'PALLIATES', 'LOCALIZES' ] 

In [43]:
for index, row in df_edges.iterrows():
    
    if row["metaedge"] == relations[0]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[3] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )
        
        
    elif row["metaedge"] == relations[1]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[8] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )


    elif row["metaedge"] == relations[2]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[1] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )


    elif row["metaedge"] == relations[3]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[11] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[4]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[9] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[5]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[7] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )


    elif row["metaedge"] == relations[6]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[0] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[7]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[1] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[8]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[8] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[9]:
      # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[6] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )


    elif row["metaedge"] == relations[10]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[10] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[11]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[1] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[12]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[0] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[13]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[4] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[14]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[5] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )


    elif row["metaedge"] == relations[15]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[0] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )

    elif row["metaedge"] == relations[16]:
        # Search for first kind by id
        query = {
            "$and" : [
                {"id" : row["ource"]},
            ]
        }
        result = list(collection.find(query))

        if len(result) == 0:
            print("ERROR: NOT FOUND.", row)
            continue

        # Relationship
        rel_property = {
            "$push" : { Relationships[2] : row["target"] }
        }

        # Add to MonngoDB
        for doc in result:
            collection.update_many( {"id" : row["ource"]}, rel_property )


In [52]:
# Number of Relationships
total_size = 0

# Iterate through documents in the collection
for document in collection.find():
    for field in Relationships:
        # Check if the field exists
        if field in document:
            # Calculate the size of the array and add it to the total size
            total_size += len(document[field])

print("Total size of arrays:", total_size)

Total size of arrays: 1292210


In [13]:
# Question 1 : Based on a disease, what compounds treats or palliates the disease?
singular_disease_id = "Disease::DOID:9074"  # Replace with your specific disease id

# Query to find compounds that TREATS or PALLIATES the disease
pipeline = [
    {
        "$match": {
            "kind": "Compound",
            "$or": [
                {
                    "TREATS": 
                    {
                        "$in": [singular_disease_id, None]
                    }
                },
                {
                    "PALLIATES": 
                    {
                        "$in": [singular_disease_id, None]
                    }
                }
                    ]
                    }
    },
    {
        "$project": {
            "_id": 0,
            "Compound Name": "$name",
            "TREATS Disease": {
                "$cond": [
                    { "$and": [
                        { "$eq": ["$kind", "Compound"] },
                        { "$ifNull": ["$TREATS", False] }
                    ] },
                    { "$in": [singular_disease_id, "$TREATS"] },
                    False
                ]
            },
            "PALLIATES Disease": {
                "$cond": [
                    { "$and": [
                        { "$eq": ["$kind", "Compound"] },
                        { "$ifNull": ["$PALLIATES", False] }
                    ] },
                    { "$in": [singular_disease_id, "$PALLIATES"] },
                    False
                ]
            }
        }
    }
]

result = collection.aggregate(pipeline)

# Query to retrieve the disease name based on singular_disease_id
disease_query = {
    "id": singular_disease_id
}

disease = collection.find_one(disease_query)

# Print the disease name
if disease:
    disease_name = disease.get("name")
    localized_to = disease.get("LOCALIZES")
    if disease_name:
        print("Disease Name:", disease_name)
    if localized_to:
        print("Located: ")
        for location in localized_to:
            anatomy_query = {
                "id" : location
            }
            loc = collection.find_one(anatomy_query)
            anatomy_name = loc.get("name")
            print(anatomy_name, end=", ")
            

# Print compounds that TREATS or PALLIATES the disease
for compound_info in result:
    compound_name = compound_info['Compound Name']
    treats_disease = compound_info['TREATS Disease']
    palliates_disease = compound_info['PALLIATES Disease']

    if treats_disease or palliates_disease:
        print("\n\nCompound Name:", compound_name)
        if treats_disease:
            print("TREATS this disease.")
        if palliates_disease:
            print("PALLIATES this disease.")


Disease Name: systemic lupus erythematosus
Located: 
manual digit, central nervous system, elastic tissue, synovial fluid, diaphragm, sacro-iliac joint, nail, tunica intima, skin of body, cardial valve, carpal region, pleura, aortic valve, lymphoid tissue, connective tissue, tendon, immune system, tunica media, bone marrow, kidney, atrioventricular node, urine, alveolus of lung, mitral valve, musculoskeletal system, umbilical vein, nervous system, cortex of kidney, basal lamina of epithelium, central retinal vein, choroid plexus, renal vein, nephron tubule, blood plasma, endothelium, lung, 

Compound Name: Cyclosporine
TREATS this disease.


Compound Name: Pregabalin
PALLIATES this disease.


Compound Name: Dapsone
TREATS this disease.


Compound Name: Amitriptyline
PALLIATES this disease.


Compound Name: Betamethasone
TREATS this disease.


Compound Name: Celecoxib
PALLIATES this disease.


Compound Name: Methotrexate
TREATS this disease.


Compound Name: Triamcinolone
TREATS this di

In [12]:
# Question 2 Logic : What new compounds can treat or palliate a disease that were recently not know to interact with the disease?
#If kind : “Anatomy” UPREGULATES and kind : “Disease” DOWNREGULATES the same gene OR
# kind : "Anatomy DOWNREGULATES" and kind : "Disease" UPREGULATES, then check 
# if this gene also INTERACTS, COVARIES, or REGULATES with another gene. 
#Then, check which Disease ASSOCIATES with those genes.
#Check that those Disease are not TREATED or PALLIATED by a Compound. 
#If all true, return Compound, Disease pair in a new array.
